In [ ]:
import pandas as pd
from epilepsy_prediction import load_prediction_model, data_preprocess,fuse_string,evaluate_model,load_imputation_model
from epilepsy_prediction.imputation import column_imputer
from copy import deepcopy

# How to predict

Loading the model, we can select between: Carbamazepine, Gabapentin, Lamotrigine, Levetiracetam, Oxcarbazepine, Phenytoin,Topiramate,Valproate. 

In [ ]:
def load_data(change_name_file,data_file_name,outcome_col="outcome"):
    data_df = pd.read_csv(data_file_name, index_col=0)
    new_name_df = pd.read_csv(change_name_file,index_col=0)
    data_df.rename(new_name_df['marketscan'].to_dict(),inplace=True,axis=1)
    y_train = data_df.loc[data_df["test_train"]=="train",outcome_col]
    X_train = data_df.loc[data_df["test_train"]=="train",~( data_df.columns.isin([outcome_col,"test_train"]))]
    y_test = data_df.loc[~(data_df["test_train"]=="train"),outcome_col]
    X_test = data_df.loc[~(data_df["test_train"]=="train"),~ (data_df.columns.isin([outcome_col,"test_train"]))]
    return X_train,y_train,X_test,y_test


In [ ]:
change_name_file=r"C:\\Users\Anastassia Kolde\\Documents\\Israel secondment\\dictionary.csv"
data_file_name = r"C:\\Users\Anastassia Kolde\\Documents\\Israel secondment\\Data\test_train_epilepsy.csv"
X_train,y_train,X_test,y_test = load_data(change_name_file,data_file_name)

In [ ]:
drug_name="carbamazepine"
clf = load_prediction_model(drug_name)
X = data_preprocess(X_train,impute_drug=drug_name,replace_mean=True)
Xs = data_preprocess(X_test,impute_drug=drug_name,replace_mean=True)

And we can also use the model for training using it's fit method. 

In [ ]:
new_clf = deepcopy(clf)
new_clf.fit(X,y_train)
new_clf.save_model('../models/prediction/est_carba_p.xgb')

In [ ]:
eval = evaluate_model(clf,new_clf,Xs,y_test)
rep = fuse_string(eval)

In [ ]:
print(rep)

# new outcome

In [ ]:
data_file_name = r"C:\\Users\Anastassia Kolde\\Documents\\Israel secondment\\Data\test_train_epilepsy.csv"
X_train,y_train,X_test,y_test = load_data(change_name_file,data_file_name)
drug_name="carbamazepine"
clf = load_prediction_model(drug_name)
X = data_preprocess(X_train,impute_drug=drug_name)
Xs = data_preprocess(X_test,impute_drug=drug_name)
new_clf = deepcopy(clf)
new_clf.fit(X,y_train)
new_clf.save_model('../models/prediction/est_carba_new_out.xgb')
eval = evaluate_model(clf,new_clf,Xs,y_test)
rep = fuse_string(eval)

In [ ]:
print(rep)

# Using the regressor imputation

In [ ]:
#col_imputer = load_imputation_model("col_carbamazepine")
regress_imputer = load_imputation_model("regress_imputer_carbamazepine.pkl")

In [ ]:
X = data_preprocess(X_train,impute_drug=drug_name,replace_mean=False)
Xs = data_preprocess(X_test,impute_drug=drug_name,replace_mean=False)
print('Do we have nan value !:',any(X.isnull()))
X = regress_imputer.transform(X)
Xs = regress_imputer.transform(Xs)
new_clf = deepcopy(clf)
new_clf.fit(X,y_train)
new_clf.save_model('../models/prediction/est_carba_new_impute.xgb')
eval = evaluate_model(clf,new_clf,Xs,y_test)
rep = fuse_string(eval)

In [ ]:
print(rep)